In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas_ml_quant
import pandas_quant_data_provider as qd

pandas_ta_quant_plot not found: No module named 'mplcursors'
automatically imported pandas_quant_data_provider 0.2.0


In [2]:
print(pandas_ml_quant.__version__)
df = qd.fetch("SPY")

0.2.0


In [3]:
with df.model() as m:
    from sklearn.neural_network import MLPClassifier

    from pandas_ml_common import random_splitter
    from pandas_ml_common.sampling.cross_validation import KEquallyWeightEvents
    from pandas_ml_utils import SkModel, PostProcessedFeaturesAndLabels, ClassificationSummary

    fit = m.fit(
        SkModel(
            MLPClassifier(activation='tanh', hidden_layer_sizes=(60, 50), random_state=42, warm_start=True, max_iter=2),
            PostProcessedFeaturesAndLabels(
                features=
                    [
                        lambda df: df["Close"].ta.macd()._[['macd.*', 'signal.*']],
                        lambda df: df.ta.adx()._[['+DI', '-DM', '+DM']],
                        lambda df: df["Close"].ta.mom(),
                        lambda df: df["Close"].ta.apo(),
                        lambda df: df.ta.atr(),
                        lambda df: df["Close"].ta.trix(),
                    ],
                feature_post_processor=lambda df: df.ta.rnn(280),
                labels=[
                    lambda df: df["Close"].ta.future_bband_quantile(include_mean=False).ta.one_hot_encode_discrete()
                ],
                targets=[
                    lambda df: df["Close"].ta.bbands()[["lower", "upper"]]
                ]
            ),
            summary_provider=ClassificationSummary,
        ),
        random_splitter(test_size=0.4, seed=42),
        cross_validation=KEquallyWeightEvents(n_splits=3)
    )

fit

exc_type: <class 'AttributeError'>
exc_value: 'function' object has no attribute 'splitter'
exc_traceback: <traceback object at 0x7f2050603c00>


AttributeError: 'function' object has no attribute 'splitter'

In [ ]:
fit.model._history